## Working with Open-EAGGR ISEA4T in Vgrid DGGS

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.vgrid.vn/lab/index.html?path=vgrid/05_isea4t.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/main/docs/notebooks/05_isea4t.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD?filepath=docs/notebooks/05_isea4t.ipynb)
[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeoshub/vgrid/blob/main/docs/notebooks/05_isea4t.ipynb)
[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/05_isea4t.ipynb)

Full Vgrid DGGS documentation is available at [vgrid document](https://vgrid.gishub.vn).

To work with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/). Full Vgridpandas DGGS documentation is available at [vgridpandas document](https://vgridpandas.gishub.vn).

To work with Vgrid DGGS in QGIS, install the [Vgrid Plugin](https://plugins.qgis.org/plugins/vgridtools/).

To visualize DGGS in Maplibre GL JS, try the [vgrid-maplibre](https://www.npmjs.com/package/vgrid-maplibre) library.

For an interactive demo, visit the [Vgrid Homepage](https://vgrid.vn).

### Install vgrid
Uncomment the following line to install [vgrid](https://pypi.org/project/vgrid/).

In [ ]:
# %pip install vgrid --upgrade

### latlon2isea4t

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2isea4t

lat = 10.775276
lon = 106.706797
res = 12
isea4t_id = latlon2isea4t(lat, lon, res)
isea4t_id

### ISEA4T to Polygon

In [ ]:
from vgrid.conversion.dggs2geo.isea4t2geo import isea4t2geo

isea4t_geo = isea4t2geo(isea4t_id)
isea4t_geo

### ISEA4T to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.isea4t2geo import isea4t2geojson

isea4t_geojson = isea4t2geojson(isea4t_id)
isea4t_geojson

### Vector to ISEA4T

In [ ]:
from vgrid.conversion.vector2dggs.vector2isea4t import vector2isea4t

file_path = "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon2.geojson"
vector_to_isea4t = vector2isea4t(
    file_path,
    resolution=16,
    compact=False,
    topology=True,
    predicate="intersects",
    output_format="gpd",
)
# Visualize the output
vector_to_isea4t.plot()
# vector_to_isea4t.plot(edgecolor="white")

### ISEA4T Compact

In [ ]:
from vgrid.conversion.dggscompact.isea4tcompact import isea4tcompact

isea4t_compacted = isea4tcompact(vector_to_isea4t, output_format="gpd")
isea4t_compacted.plot(edgecolor="white")

### ISEA4T Expand

In [ ]:
from vgrid.conversion.dggscompact.isea4tcompact import isea4texpand

isea4t_expanded = isea4texpand(vector_to_isea4t, resolution=17, output_format="gpd")
isea4t_expanded.plot(edgecolor="white")

### ISEA4T Binning

In [ ]:
from vgrid.binning.isea4tbin import isea4tbin

file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/dist1_pois.csv"
)
stats = "count"
isea4t_bin = isea4tbin(
    file_path,
    resolution=15,
    stats=stats,
    # numeric_field="confidence",
    # category="category",
    output_format="gpd",
)
isea4t_bin.plot(
    column=stats,  # numeric column to base the colors on
    cmap="Spectral_r",  # color scheme (matplotlib colormap)
    legend=True,
    linewidth=0.2,  # boundary width (optional)
)

### Raster to ISEA4T

#### Download and open raster

In [ ]:
from vgrid.utils.io import download_file
import rasterio
from rasterio.plot import show

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/rgb.tif"
)
raster_file = download_file(raster_url)
src = rasterio.open(raster_file, "r")
print(src.meta)
show(src)

#### Convert raster to ISEA4T

In [ ]:
# %pip install folium

In [ ]:
from vgrid.conversion.raster2dggs.raster2isea4t import raster2isea4t
from vgrid.utils.io import download_file

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/rgb.tif"
)
raster_file = download_file(raster_url)

raster_to_isea4t = raster2isea4t(raster_file, output_format="gpd")

# Visualize the output
import folium

m = folium.Map(tiles="CartoDB positron", max_zoom=28)

isea4t_layer = folium.GeoJson(
    raster_to_isea4t,
    style_function=lambda x: {
        "fillColor": f"rgb({x['properties']['band_1']}, {x['properties']['band_2']}, {x['properties']['band_3']})",
        "fillOpacity": 1,
        "color": "black",
        "weight": 1,
    },
    popup=folium.GeoJsonPopup(
        fields=["isea4t", "band_1", "band_2", "band_3"],
        aliases=["ISEA4T ID", "Band 1", "Band 2", "Band 3"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(isea4t_layer.get_bounds())

# Display the map
m

### ISEA4T Generator

In [ ]:
from vgrid.generator.isea4tgrid import isea4tgrid

# isea4t_grid = isea4tgrid(resolution=1, output_format="gpd")
isea4t_grid = isea4tgrid(resolution=16,bbox=[106.699007, 10.762811, 106.717674, 10.778649],output_format="gpd")
# isea4t_grid = isea4tgrid(resolution=6,bbox=[93.83597510,5.17138775,119.49128538,22.19744870],output_format="gpd") 
isea4t_grid.plot(edgecolor="white")

### ISEA4T Inspect

In [ ]:
from vgrid.stats.isea4tstats import isea4tinspect

resolution = 5
isea4t_inspect = isea4tinspect(resolution)
isea4t_inspect.head()

### ISEA4T Normalized Area Histogram

In [ ]:
from vgrid.stats.isea4tstats import isea4t_norm_area_hist

isea4t_norm_area_hist(isea4t_inspect)

### Distribution of ISEA4T Area Distortions

In [ ]:
from vgrid.stats.isea4tstats import isea4t_norm_area

isea4t_norm_area(isea4t_inspect)

### ISEA4T IPQ Compactness Histogram

Isoperimetric Inequality (IPQ) Compactness (suggested by [Osserman, 1978](https://sites.math.washington.edu/~toro/Courses/20-21/MSF/osserman.pdf)):

$$C_{IPQ} = \frac{4 \pi A}{p^2}$$
The range of the IPQ compactness metric is [0,1]. 

A circle represents the maximum compactness with a value of 1. 

As shapes become more irregular or elongated, their compactness decreases toward 0.

In [ ]:
from vgrid.stats.isea4tstats import isea4t_compactness_ipq_hist

isea4t_compactness_ipq_hist(isea4t_inspect)

### Distribution of ISEA4T IPQ Compactness

In [ ]:
from vgrid.stats.isea4tstats import isea4t_compactness_ipq

isea4t_compactness_ipq(isea4t_inspect)

### ISEA4T Convex hull Compactness Histogram:

$$C_{CVH} = \frac{A}{A_{CVH}}$$


The range of the convex hull compactness metric is [0,1]. 

As shapes become more concave, their convex hull compactness decreases toward 0.

In [ ]:
from vgrid.stats.isea4tstats import isea4t_compactness_cvh_hist
isea4t_compactness_cvh_hist(isea4t_inspect)

### Distribution of ISEA4T Convex hull Compactness

In [ ]:
from vgrid.stats.isea4tstats import isea4t_compactness_cvh
isea4t_compactness_cvh(isea4t_inspect)

### ISEA4T Statistics

Characteristic Length Scale (CLS - suggested by Ralph Kahn): the diameter of a spherical cap of the same cell's area

In [ ]:
from vgrid.stats.isea4tstats import isea4tstats

isea4tstats()